In [ ]:
import sys
path = '/Users/xiehao/Desktop/workspace/X2AI/sugar_ai/'
if path not in sys.path:
    sys.path.append(path)

import pandas as pd
import numpy as np
from base import DBFile, DBSQL


# ==================== 趋势类指标 ====================

# 1. MA (简单移动平均和指数移动平均)
def calculate_ma(df: pd.DataFrame, fields: list):
    # 计算均线
    for window in [5, 10, 20, 30, 60, 120]:
        df[f'sma_{window}d'] = df['close'].rolling(window=window).mean()
        df[f'ema_{window}d'] = df['close'].ewm(span=window, adjust=False).mean()
        fields.append(f'sma_{window}d')
        fields.append(f'ema_{window}d')
    return df, fields

# 2. MACD
def calculate_macd(df, fast=12, slow=26, signal=9):
    df['EMA_fast'] = df['Close'].ewm(span=fast, adjust=False).mean()
    df['EMA_slow'] = df['Close'].ewm(span=slow, adjust=False).mean()
    df['DIF'] = df['EMA_fast'] - df['EMA_slow']
    df['DEA'] = df['DIF'].ewm(span=signal, adjust=False).mean()
    df['MACD'] = 2 * (df['DIF'] - df['DEA'])
    return df

# 3. DMI (简化版)
def calculate_dmi(df, window=14):
    df['TR'] = df['High'] - df['Low']
    df['+DM'] = np.where((df['High'].diff() > df['Low'].diff().abs()), df['High'].diff(), 0)
    df['-DM'] = np.where((df['Low'].diff().abs() > df['High'].diff()), df['Low'].diff().abs(), 0)
    df['+DI'] = df['+DM'].rolling(window).mean() / df['TR'].rolling(window).mean() * 100
    df['-DI'] = df['-DM'].rolling(window).mean() / df['TR'].rolling(window).mean() * 100
    df['ADX'] = abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI']) * 100
    df['ADX'] = df['ADX'].rolling(window).mean()
    return df

# ==================== 动量类指标 ====================

# 4. RSI
def calculate_rsi(df, window=14):
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# 5. CCI
def calculate_cci(df, window=20):
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['MA_TP'] = df['TP'].rolling(window).mean()
    df['MeanDev'] = abs(df['TP'] - df['MA_TP']).rolling(window).mean()
    df['CCI'] = (df['TP'] - df['MA_TP']) / (0.015 * df['MeanDev'])
    return df

# 6. ROC
def calculate_roc(df, window=12):
    df['ROC'] = (df['Close'] - df['Close'].shift(window)) / df['Close'].shift(window) * 100
    return df

# ==================== 震荡类指标 ====================

# 7. KDJ
def calculate_kdj(df, window=9):
    low_min = df['Low'].rolling(window).min()
    high_max = df['High'].rolling(window).max()
    df['RSV'] = (df['Close'] - low_min) / (high_max - low_min) * 100
    df['K'] = df['RSV'].ewm(alpha=1/3, adjust=False).mean()
    df['D'] = df['K'].ewm(alpha=1/3, adjust=False).mean()
    df['J'] = 3 * df['K'] - 2 * df['D']
    return df

# 8. BOLL
def calculate_boll(df, window=20, k=2):
    df['MA'] = df['Close'].rolling(window).mean()
    df['STD'] = df['Close'].rolling(window).std()
    df['Upper'] = df['MA'] + k * df['STD']
    df['Lower'] = df['MA'] - k * df['STD']
    return df

# 9. ATR
def calculate_atr(df, window=14):
    df['TR1'] = df['High'] - df['Low']
    df['TR2'] = abs(df['High'] - df['Close'].shift())
    df['TR3'] = abs(df['Low'] - df['Close'].shift())
    df['TR'] = df[['TR1', 'TR2', 'TR3']].max(axis=1)
    df['ATR'] = df['TR'].ewm(span=window, adjust=False).mean()
    return df

# ==================== 成交量类指标 ====================

# 10. OBV
def calculate_obv(df):
    df['OBV'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], 
                         np.where(df['Close'] < df['Close'].shift(1), -df['Volume'], 0)).cumsum()
    return df

# 11. VOL (简单均量线)
def calculate_vol(df, window=5):
    df['MAVOL'] = df['Volume'].rolling(window).mean()
    return df

# 12. MFI
def calculate_mfi(df, window=14):
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['MoneyFlow'] = df['TP'] * df['Volume']
    df['PositiveFlow'] = np.where(df['TP'] > df['TP'].shift(1), df['MoneyFlow'], 0)
    df['NegativeFlow'] = np.where(df['TP'] < df['TP'].shift(1), df['MoneyFlow'], 0)
    pos_flow = df['PositiveFlow'].rolling(window).sum()
    neg_flow = df['NegativeFlow'].rolling(window).sum()
    df['MFI'] = 100 - (100 / (1 + (pos_flow / neg_flow)))
    return df

# ==================== 价格行为类指标 ====================

# 13. PSY
def calculate_psy(df, window=12):
    df['Up'] = np.where(df['Close'] > df['Close'].shift(1), 1, 0)
    df['PSY'] = df['Up'].rolling(window).sum() / window * 100
    return df

# 14. WR
def calculate_wr(df, window=14):
    df['Highest'] = df['High'].rolling(window).max()
    df['Lowest'] = df['Low'].rolling(window).min()
    df['WR'] = (df['Highest'] - df['Close']) / (df['Highest'] - df['Lowest']) * -100
    return df

# 15. SAR (简化版)
def calculate_sar(df, af=0.02, max_af=0.2):
    # 简化实现，真实SAR计算更复杂
    df['SAR'] = np.nan
    df.loc[0, 'SAR'] = df.loc[0, 'Low']
    for i in range(1, len(df)):
        if df.loc[i, 'Close'] > df.loc[i-1, 'Close']:
            df.loc[i, 'SAR'] = df.loc[i-1, 'SAR'] + af * (df.loc[i-1, 'High'] - df.loc[i-1, 'SAR'])
        else:
            df.loc[i, 'SAR'] = df.loc[i-1, 'SAR'] - af * (df.loc[i-1, 'SAR'] - df.loc[i-1, 'Low'])
    return df


In [ ]:
db = DBFile()
df = db.read_data(
    table="future_bar1d",
    filters={"date": ["2024-01-01", "2025-06-09"]}
)
fields = []

# 1. MA (简单移动平均和指数移动平均)
def calculate_sma(df: pd.DataFrame):
    # 计算均线
    result = []
    for window in [5, 10, 20, 30, 60, 120]:
        df["sma"] = df['close'].rolling(window=window).mean()
        df["param"] = window
        result.append(df[["date", "instrument", "param", "sma"]])
    return result

result = calculate_sma(df)
pd.concat(result).sort_values(["date", "param"])